# 문제 6

[Kaggle 형] train_prob.csv로 failure 예측하는 모델을 만들고, 

test_prob.csv에 대한 failure가 1일 확률 예측하여 다음과 같은 형식의 answer6.csv를 만들어라. 

측정 지표는 AUC(area under of ROC curve)이다. id 는 테스트 케이스의 id 이고, failure에는 failure가 1이 될 확률이다.

id,failure

16115, 0.1

16116, 0.2


**강사: 멀티캠퍼스 강선구(sunku0316.kang@multicampus.com, sun9sun9@gmail.com)**

# Kaggle형 풀이 단계

Step 0: Kaggle용 데이터셋을 만든다.

Step 1: 검증 방법을 정하고, 검증 루틴을 만듭니다.

Step 2: Baseline 모델을 만듭니다

Step 3: 모델 선택 루틴을 만듭니다.

|id|failure|
|----|----|
|16115| 0.1|
|16116| 0.2|

....	

Step 4: 모델 개선 작업을 합니다.

In [1]:
# 실행 환경 확인

import pandas as pd
import numpy as np
import sklearn
import scipy
import statsmodels
import mlxtend
import sys
import xgboost as xgb

print(sys.version)
for i in [pd, np, sklearn, scipy, mlxtend, statsmodels, xgb]:
    print(i.__name__, i.__version__)

3.7.4 (tags/v3.7.4:e09359112e, Jul  8 2019, 20:34:20) [MSC v.1916 64 bit (AMD64)]
pandas 0.25.1
numpy 1.18.5
sklearn 0.21.3
scipy 1.5.2
mlxtend 0.15.0.0
statsmodels 0.11.1
xgboost 0.80


In [2]:
df_train = pd.read_csv('train_prob.csv', index_col = ['id'])
df_test = pd.read_csv('test_prob.csv', index_col = ['id'])
# 실제시험 x
s_kaggle_ans = pd.read_csv('test_prob_ans.csv', index_col = ['id'])['failure']

In [3]:
df_train = df_train.assign(
    na_1 = lambda x: x['measurement_3'].isna(),
    na_2 = lambda x: x['measurement_5'].isna(),
)
df_test = df_test.assign(
    na_1 = lambda x: x['measurement_3'].isna(),
    na_2 = lambda x: x['measurement_5'].isna(),
)

In [4]:
df_train['product_code'].value_counts()

C    5765
E    5343
B    5250
A    5100
Name: product_code, dtype: int64

In [5]:
df_test['product_code'].value_counts()

D    5112
Name: product_code, dtype: int64

In [7]:
# 방법 1: 따로따로
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.linear_model import LinearRegression

imp = IterativeImputer(
    LinearRegression(fit_intercept=True), # 𝑚𝑒𝑎𝑠𝑢𝑟𝑒𝑚𝑒𝑛𝑡17=𝛽0+𝛽3𝑚𝑒𝑎𝑠𝑢𝑟𝑒𝑚𝑒𝑛𝑡3+𝛽4𝑚𝑒𝑎𝑠𝑢𝑟𝑒𝑚𝑒𝑛𝑡4+...+𝛽9𝑚𝑒𝑎𝑠𝑢𝑟𝑒𝑚𝑒𝑛𝑡9
    random_state=123
)
# product_code 별
X_imp = ['measurement_{}'.format(i) for i in range(3, 10)] + ['measurement_17']
df_train[X_imp] = df_train.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index =x.index, columns = X_imp)
)
df_test[X_imp] = df_test.groupby('product_code')[X_imp].apply(
    lambda x: pd.DataFrame(imp.fit_transform(x), index =x.index, columns = X_imp)
)
X_mean = ['measurement_{}'.format(i) for i in range(10, 17)]
df_train[X_mean] = df_train.groupby('product_code')[X_mean].transform(lambda x: x.fillna(x.mean()))
df_test[X_mean] = df_test.groupby('product_code')[X_mean].transform(lambda x: x.fillna(x.mean()))

- attribute_0와 attribute_1 은 유용하지 않다.
- product_code에 따라 loding_log의 평균 같다.
- loading + log -> 대칭형
- std: \['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17'\] + pt: ['na_1'] -> LR
- std: \['measurement_0~17\] -> LDA transformer
- LDA: predictor
- std: \['measurement_0~17\] -> PCA n_components = 7
- rf: {'max_depth': [5, 6, 7], 'min_samples_split': [256, 512], 'n_estimators': [5, 10, 15]}

In [10]:
m = pd.concat([df_train['loading'], df_test['loading']]).mean()
df_train['loading'] = df_train['loading'].fillna(m)
df_test['loading'] = df_test['loading'].fillna(m)

In [13]:
df_train.isna().sum().sum(), df_test.isna().sum().sum()

(0, 0)

In [14]:
X_all = df_test.columns.tolist()
np.array(X_all)

array(['product_code', 'loading', 'attribute_0', 'attribute_1',
       'attribute_2', 'attribute_3', 'measurement_0', 'measurement_1',
       'measurement_2', 'measurement_3', 'measurement_4', 'measurement_5',
       'measurement_6', 'measurement_7', 'measurement_8', 'measurement_9',
       'measurement_10', 'measurement_11', 'measurement_12',
       'measurement_13', 'measurement_14', 'measurement_15',
       'measurement_16', 'measurement_17', 'na_1', 'na_2'], dtype='<U14')

In [15]:
df_train['failure'].value_counts(normalize=  True)

0    0.788564
1    0.211436
Name: failure, dtype: float64

# Step1: 검증 방법을 정하고, 검증 루틴을 만듭니다.

Day1에서 했던 Holdout 검증보다 더 안정적인 평가 결과를 얻을 수 있는 교차 검증을 해봅니다.

In [17]:
from sklearn.model_selection import GroupKFold
gkf = GroupKFold(4)

for train_idx, valid_idx in gkf.split(df_train[X_all], df_train['failure'], groups = df_train['product_code']):
    print(df_train.iloc[train_idx]['product_code'].unique(), df_train.iloc[valid_idx]['product_code'].unique())

['A' 'B' 'E'] ['C']
['A' 'B' 'C'] ['E']
['A' 'C' 'E'] ['B']
['B' 'C' 'E'] ['A']


In [ ]:
def eval_model(model_name, clf):
    """
        모델의 인스턴스를 받아 검증 결과(AUC) 를 구합니다.
        1. GroupKFold 검증 - df_train, groups - product_code
        2. df_train 대한 교차검증결과에 대한 AUC를 구합니다.
        3. 주어진 모델명으로 평가 결과를 저장합니다. Format: Valid: {:.5f}±{:.5f}, Train: {:.5f}±{:.5f}
        4. 가장 최근의 수행결과를 보여 주어 선택하는데 활용하도록 합니다.
    Parameters:
        model_name: str, 모델의 이름,
        clf: sklearn object, 모델 인스턴스
    """
    
    return ""

# 모델 선택 루틴입니다. (Step 3)
def select_model(clf):
    """
        1. 전체 학습데이터(df_train)로 학습을 합니다. 
        2. df_test에 대한 예측을 합니다.
        3. 예측 결과를 출력양식(id, failure)에 맞춰 csv파일을 만듭니다.
        4. 자가 채점을 위한 예측 결과를 반환합니다.
    Parameters: 
        clf: sklearn object, 모델 인스턴스
    Returns: 1차원 np.ndarray
        예측 결과
    """
    
    return prd

In [19]:
from sklearn.model_selection import cross_validate
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import LogisticRegression
# std: ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17'] + pt: ['na_1'] -> LR
ct = ColumnTransformer([
    ('std', StandardScaler(), ['loading', 'measurement_1', 'measurement_4', 'measurement_14', 'measurement_17']),
    ('pt', 'passthrough', ['na_1'])
])
clf_lr = make_pipeline(ct, LogisticRegression(solver = 'lbfgs'))
cross_validate(
    clf_lr, df_train[X_all], df_train['failure'], groups = df_train['product_code'], scoring = 'roc_auc',
    cv = gkf, return_train_score = True
)

{'fit_time': array([0.06014419, 0.06818438, 0.04991698, 0.05030346]),
 'score_time': array([0.01984668, 0.01216531, 0.02041149, 0.00995398]),
 'test_score': array([0.58821746, 0.58491734, 0.58894014, 0.59540058]),
 'train_score': array([0.59262252, 0.59350804, 0.59192438, 0.58956303])}